# Key Takeaways

Chapter 4 of Wes McKinney's [*Python for Data Analysis*](https://wesmckinney.com/pages/book.html) discusses the NumPy package (short for numerical Python), which is the foundation for numerical computing in Python, including pandas.

***The key takeaways from this notebook are:***

1. Creating arrays
1. Slicing arrays
1. Performing mathematical operations on arrays
1. Applying functions and methods to arrays
1. Using conditional logic with arrays (i.e., `np.where()` and `np.select()`)

***Note:*** Indented block quotes are from McKinney unless otherwise indicated. The section numbers here may differ from McKinney because we may not discuss every topic.

The typical abbreviation for NumPy is `np`.

In [1]:
import numpy as np

The following prints NumPy arrays to 4 decimals places without changing the precision of the underlying array.

In [2]:
np.set_printoptions(precision=4, suppress=True)

NumPy is critical to numerical computing in Python.
We will often use NumPy via McKinney's pandas, but data analysts must know the fundamentals of NumPy.

> NumPy, short for Numerical Python, is one of the most important foundational packages for numerical computing in Python. Most computational packages providing scientific functionality use NumPy’s array objects as the lingua franca for data exchange.
>
> Here are some of the things you’ll find in NumPy:
> - ndarray, an efficient multidimensional array providing fast array-oriented arithmetic operations and flexible broadcasting capabilities.
> - Mathematical functions for fast operations on entire arrays of data without having to write loops.
> - Tools for reading/writing array data to disk and working with memory-mapped files.
> - Linear algebra, random number generation, and Fourier transform capabilities.
> - A C API for connecting NumPy with libraries written in C, C++, or FORTRAN.
>
> Because NumPy provides an easy-to-use C API, it is straightforward to pass data to external libraries written in a low-level language and also for external libraries to return data to Python as NumPy arrays. This feature has made Python a language of choice for wrapping legacy C/C++/Fortran codebases and giving them a dynamic and
easy-to-use interface. 
>
> While NumPy by itself does not provide modeling or scientific functionality, having an understanding of NumPy arrays and array-oriented computing will help you use tools with array-oriented semantics, like pandas, much more effectively. Since NumPy is a large topic, I will cover many advanced NumPy features like broadcasting
in more depth later (see Appendix A). For most data analysis applications, the main areas of functionality I’ll focus on are: 
> - Fast vectorized array operations for data munging and cleaning, subsetting and filtering, transformation, and any other kinds of computations
> - Common array algorithms like sorting, unique, and set operations
> - Efficient descriptive statistics and aggregating/summarizing data
> - Data alignment and relational data manipulations for merging and joining together heterogeneous datasets
> - Expressing conditional logic as array expressions instead of loops with if-elif-else branches
> - Group-wise data manipulations (aggregation, transformation, function application)
>
> While NumPy provides a computational foundation for general numerical data processing, many readers will want to use pandas as the basis for most kinds of statistics or analytics, especially on tabular data. pandas also provides some more domain-specific functionality like time series manipulation, which is not present in NumPy.
>
> One of the reasons NumPy is so important for numerical computations in Python is because it is designed for efficiency on large arrays of data. There are a number of reasons for this:
> - NumPy internally stores data in a contiguous block of memory, independent of other built-in Python objects. NumPy’s library of algorithms written in the C language can operate on this memory without any type checking or other overhead. NumPy arrays also use much less memory than built-in Python sequences.
> - NumPy operations perform complex computations on entire arrays without the need for Python for loops.


McKinney provides a clear example of NumPy's power and speed.
He creates 2 sequences of numbers from 0 to 999,999 as a NumPy array and a Python list, then multiplies the list by 2.
The NumPy array supports vectorized operations and it "just works" when he multiplies the array by 2.
However, he must use a list comprehension to multiply the list by 2 element-by-element.

In [3]:
my_list = list(range(1000000))

In [4]:
my_arr = np.arange(1000000)

In [5]:
my_list[:5]

[0, 1, 2, 3, 4]

In [6]:
my_arr[:5]

array([0, 1, 2, 3, 4])

Multiplying lists by an integer concatenates them instead of elementwise multiplication.
So we use a list comprehension to multiply the elements in `my_list` by 2.

In [7]:
# my_list * 2 # concatenates two copies of my_list

In [8]:
# [2 * x for x in my_list] # we use a list comprehension for elementwise multiplication of a list

However, math on NumPy arrays "just works".

In [9]:
my_arr * 2

array([      0,       2,       4, ..., 1999994, 1999996, 1999998])

We use the "magic" function `%timeit` to time these two calculations.

In [10]:
%timeit [x * 2 for x in my_list]

107 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit my_arr * 2

2.17 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The NumPy version is few hundred times faster than the list version.

# The NumPy ndarray: A Multidimensional Array Object

> One of the key features of NumPy is its N-dimensional array object, or ndarray, which is a fast, flexible container for large datasets in Python. Arrays enable you to perform mathematical operations on whole blocks of data using similar syntax to the equivalent operations between scalar elements.

We can generate random data to explore NumPy arrays.
Whenever we use random data, we should set the random number seed so that our random numbers are repeatable.
Because we set the random number seed just before we generate `data`, our `data`s will be identical.

In [12]:
np.random.seed(42) # to make your code/analysis, always set the random number seed
data = np.random.randn(2, 3)

In [13]:
data

array([[ 0.4967, -0.1383,  0.6477],
       [ 1.523 , -0.2342, -0.2341]])

Multiplying `data` by 10 multiplies each element in `data` by 10, and adding `data` to itself does element-wise addition.
The compromise to achieve this common-sense behavior is that NumPy arrays must contain homogeneous data types (e.g., all floats or all integers).

In [14]:
data * 10

array([[ 4.9671, -1.3826,  6.4769],
       [15.2303, -2.3415, -2.3414]])

Addition in NumPy is also elementwise.

In [15]:
data + data

array([[ 0.9934, -0.2765,  1.2954],
       [ 3.0461, -0.4683, -0.4683]])

NumPy arrays also have attributes.
Recall that Jupyter Notebooks provides tab completion.

In [16]:
data.shape

(2, 3)

In [17]:
data.dtype

dtype('float64')

In [18]:
data[0]

array([ 0.4967, -0.1383,  0.6477])

In [19]:
data[0][0] # zero row, then the zero element in the zero row

0.4967141530112327

In [20]:
data[0, 0] # zero row, zero column

0.4967141530112327

## Creating ndarrays

> The easiest way to create an array is to use the array function. This accepts any sequence-like object (including other arrays) and produces a new NumPy array containing the passed data

In [21]:
data1 = [6, 7.5, 8, 0, 1]
arr1 = np.array(data1)

In [22]:
arr1

array([6. , 7.5, 8. , 0. , 1. ])

In [23]:
arr1.dtype

dtype('float64')

We could coerce these values to integers, but we would lose information.
The default is to select a `dtype` that would not lose information.

In [24]:
np.array(data1, dtype=np.int64)

array([6, 7, 8, 0, 1])

Note that `np.array()` re-cast the values in `data1` to floats becuase NumPy arrays must be homogenous data types.
A list of lists becomes a two-dimensional array.

In [25]:
data2 = [[1, 2, 3, 4], [5, 6, 7, 8]]
arr2 = np.array(data2)

In [26]:
arr2

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [27]:
arr2.ndim

2

In [28]:
arr2.shape

(2, 4)

In [29]:
arr1.dtype

dtype('float64')

In [30]:
arr2.dtype

dtype('int64')

There are several other ways to create NumPy arrays.

In [31]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
np.zeros((3, 6))

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

The `np.arange()` function is similar to the core `range()` but it creates an array directly.

In [33]:
list(range(15))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [34]:
np.arange(15)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

***Table 4-1*** lists the NumPy array creation functions.

- `array`: Convert input data (list, tuple, array, or other sequence type) to an ndarray either by inferring a dtype or explicitly specifying a dtype; copies the input data by default
- `asarray`:  Convert input to ndarray, but do not copy if the input is already an ndarray 
- `arange`:  Like the built-in range but returns an ndarray instead of a list
- `ones`, `ones_like`:  Produce an array of all 1s with the given shape and dtype; `ones_like` takes another array and produces a `ones` array of the - same shape and dtype
- `zeros`, `zeros_like`:  Like `ones` and `ones_like` but producing arrays of 0s instead
- `empty`, `empty_like`:  Create new arrays by allocating new memory, but do not populate with any values like ones and zeros
- `full`, `full_like`:  Produce an array of the given shape and dtype with all values set to the indicated "fill value"
- `eye`, `identity`:  Create a square N-by-N identity matrix (1s on the diagonal and 0s elsewhere)

---

***Practice:***
Create a 1-dimensional array named `a1` that counts from 0 to 24 by 1.

In [35]:
a1 = np.arange(0, 25)
a1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

***Practice:***
Create a 1-dimentional array named `a2` that counts from 0 to 24 by 3.

In [36]:
a2 = np.arange(0, 25, 3)
a2

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24])

***Practice:***
Create a 1-dimentional array named `a3` that counts from 0 to 100 by multiples of 3 and 5.

We can use a list comprehension for this!

In [37]:
a3 = np.array([i for i in range(101) if (i%3==0) or (i%5==0)])
a3

array([  0,   3,   5,   6,   9,  10,  12,  15,  18,  20,  21,  24,  25,
        27,  30,  33,  35,  36,  39,  40,  42,  45,  48,  50,  51,  54,
        55,  57,  60,  63,  65,  66,  69,  70,  72,  75,  78,  80,  81,
        84,  85,  87,  90,  93,  95,  96,  99, 100])

An alternative is to create an array of integers from 0 to 100, then use Booleans to slice it.
We will learn more about Boolean slices below.

In [38]:
a0100 = np.arange(101)
a0100[(a0100%3==0) | (a0100%5==0)]

array([  0,   3,   5,   6,   9,  10,  12,  15,  18,  20,  21,  24,  25,
        27,  30,  33,  35,  36,  39,  40,  42,  45,  48,  50,  51,  54,
        55,  57,  60,  63,  65,  66,  69,  70,  72,  75,  78,  80,  81,
        84,  85,  87,  90,  93,  95,  96,  99, 100])

---

## Data Types for ndarrays

NumPy provides control over data types, but you may not need this control until much later in your career when you work with larger data.

In [39]:
arr1 = np.array([1, 2, 3], dtype=np.float64)
arr2 = np.array([1, 2, 3], dtype=np.int32)

In [40]:
arr = np.array([1, 2, 3, 4, 5])
arr.dtype

dtype('int64')

In [41]:
arr = np.array([3.7, -1.2, -2.6, 0.5, 12.9, 10.1])
arr.astype(np.int32)

array([ 3, -1, -2,  0, 12, 10], dtype=int32)

## Arithmetic with NumPy Arrays

> Arrays are important because they enable you to express batch operations on data without writing any for loops. NumPy users call this vectorization. Any arithmetic operations between equal-size arrays applies the operation element-wise

In [42]:
arr = np.array([[1., 2., 3.], [4., 5., 6.]])

NumPy array addition is elementwise.

In [43]:
arr + arr

array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]])

NumPy array multiplication is elementwise.

In [44]:
arr * arr

array([[ 1.,  4.,  9.],
       [16., 25., 36.]])

NumPy array division is elementwise.

In [45]:
1 / arr

array([[1.    , 0.5   , 0.3333],
       [0.25  , 0.2   , 0.1667]])

In [46]:
arr ** 0.5

array([[1.    , 1.4142, 1.7321],
       [2.    , 2.2361, 2.4495]])

In [47]:
arr2 = np.array([[0., 4., 1.], [7., 2., 12.]])

---

***Practice:***
Create a 1-dimensional array `a3` that contains the squares of the even integers through 100,000.
How much faster is the NumPy version than the list comprehension version?

In [48]:
a3 = np.arange(100000)**2
a3

array([         0,          1,          4, ..., 9999400009, 9999600004,
       9999800001])

In [49]:
%timeit np.arange(100000, 2)**2

2.09 µs ± 427 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [50]:
%timeit np.array([i**2 for i in range(0, 100000, 2)])

27 ms ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


***Practice:***
Write functions that mimic Excel's `pv` and `fv` functions.

In [51]:
def pv(r, n, pmt, fv):
    pv_pmt = (pmt/r) * (1 - (1+r)**(-1*n))
    pv_fv = fv * (1+r)**(-1*n)
    return -1*(pv_pmt + pv_fv)

In [52]:
pv(0.1, 1, 0, 110)

-100.0

In [53]:
pv(0.1, 2, 0, 121)

-99.99999999999999

In [54]:
np.round(a=pv(0.1, 2, 0, 121), decimals=12)

-100.0

In [55]:
pv(0.05, 10, 50, 1000) # par bond with 10-year maturity and 5% annual coupon

-1000.0

In [56]:
def fv(r, n, pmt, pv):
    fv_pmt = (pmt/r) * ((1+r)**n - 1)
    fv_fv = pv * (1+r)**n
    return -1*(fv_pmt + fv_fv)

In [57]:
fv(0.1, 1, 0, -100)

110.00000000000001

In [58]:
fv(0.1, 10, 100, -1000) # par bond with 10-year maturity and 10% annual coupon

1000.0000000000002

---

## Basic Indexing and Slicing

One-dimensional array index and slice the same as lists.

In [59]:
arr = np.arange(10)

In [60]:
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [61]:
arr[5]

5

In [62]:
arr[5:8]

array([5, 6, 7])

In [63]:
equiv_list = list(range(10))

In [64]:
equiv_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [65]:
equiv_list[5:8]

[5, 6, 7]

In [66]:
# equiv_list[5:8] = 12 # TypeError: can only assign an iterable

In [67]:
equiv_list[5:8] = [12] * 3

In [68]:
equiv_list

[0, 1, 2, 3, 4, 12, 12, 12, 8, 9]

With NumPy arrays, we do not have to ump through this hoop.

In [69]:
arr[5:8] = 12
arr

array([ 0,  1,  2,  3,  4, 12, 12, 12,  8,  9])

> As you can see, if you assign a scalar value to a slice, as in `arr[5:8] = 12`, the value is propagated (or broadcasted henceforth) to the entire selection. An important first distinction from Python’s built-in lists is that array slices are views on the original array. This means that the data is not copied, and any modifications to the view will be reflected in the source array.

In [70]:
arr_slice = arr[5:8]

In [71]:
arr_slice

array([12, 12, 12])

In [72]:
arr_slice[1] = 12345

In [73]:
arr_slice

array([   12, 12345,    12])

In [74]:
arr

array([    0,     1,     2,     3,     4,    12, 12345,    12,     8,
           9])

The `:` slices every element in `arr_slice`.

In [75]:
arr_slice[:] = 64

In [76]:
arr_slice

array([64, 64, 64])

In [77]:
arr

array([ 0,  1,  2,  3,  4, 64, 64, 64,  8,  9])

> If you want a copy of a slice of an ndarray instead of a view, you will need to explicitly copy the array-for example, `arr[5:8].copy()`.

In [78]:
arr_slice_2 = arr[5:8].copy()

In [79]:
arr_slice_2

array([64, 64, 64])

In [80]:
arr_slice_2[:] = 2001

In [81]:
arr_slice_2

array([2001, 2001, 2001])

In [82]:
arr

array([ 0,  1,  2,  3,  4, 64, 64, 64,  8,  9])

> With higher dimensional arrays, you have many more options. In a two-dimensional array, the elements at each index are no longer scalars but rather one-dimensional arrays... Thus, individual elements can be accessed recursively. But that is a bit too much work, so you can pass a comma-separated list of indices to select individual elements.

In [83]:
arr2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [84]:
arr2d

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [85]:
arr2d[2]

array([7, 8, 9])

In [86]:
arr2d[0][2]

3

In [87]:
arr2d[0, 2] # row, column notation is more common and typically easier to read

3

### Indexing with slices

In [88]:
arr2d

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [89]:
arr2d[:2]

array([[1, 2, 3],
       [4, 5, 6]])

In [90]:
arr2d[:2, 1:]

array([[2, 3],
       [5, 6]])

In [91]:
arr2d[1, :2]

array([4, 5])

In [92]:
arr2d[:2, 2]

array([3, 6])

A colon (`:`) by itself selects the entire dimension and is necessary to slice higher dimensions.

In [93]:
arr2d[:, :1]

array([[1],
       [4],
       [7]])

In [94]:
arr2d[:2, 1:] = 0
arr2d

array([[1, 0, 0],
       [4, 0, 0],
       [7, 8, 9]])

***ALWAYS CHECK YOUR OUTPUT!***

## Boolean Indexing

We can use Booleans (`True`s and `False`s) to slice, too.
Think of `names` as a sequence of seven names that line up with the seven rows in `data`.
To keep things simple, we will not give column names.
The folowing example is like `index(match(), match())` from Excel.

In [95]:
names = np.array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe', 'Joe'])
np.random.seed(42)
data = np.random.randn(7, 4)

In [96]:
names

array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe', 'Joe'], dtype='<U4')

In [97]:
data

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249, -0.5623],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

In [98]:
names == 'Bob'

array([ True, False, False,  True, False, False, False])

In [99]:
data[names == 'Bob']

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [ 0.242 , -1.9133, -1.7249, -0.5623]])

In [100]:
data[names == 'Bob', 2:]

array([[ 0.6477,  1.523 ],
       [-1.7249, -0.5623]])

In [101]:
data[names == 'Bob', 3]

array([ 1.523 , -0.5623])

In [102]:
names != 'Bob'

array([False,  True,  True, False,  True,  True,  True])

The `~` inverts a Boolean condition.

In [103]:
data[~(names == 'Bob')]

array([[-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

In [104]:
cond = names == 'Bob'
data[~cond]

array([[-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

For NumPy arrays, we must use `&` and `|` instead of `and` and `or`.

In [105]:
mask = (names == 'Bob') | (names == 'Will')

In [106]:
data[mask]

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249, -0.5623],
       [-1.0128,  0.3142, -0.908 , -1.4123]])

The "not" operator is `~` in NumPy.
The "not" operator is typically `!` in other programming languages.

In [107]:
data[~(names == 'Bob')]

array([[-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

In [108]:
data

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249, -0.5623],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

In [109]:
data < 0

array([[False,  True, False, False],
       [ True,  True, False, False],
       [ True, False,  True,  True],
       [False,  True,  True,  True],
       [ True, False,  True,  True],
       [False,  True, False,  True],
       [ True, False,  True, False]])

In [110]:
data[data < 0] = 0

In [111]:
data

array([[0.4967, 0.    , 0.6477, 1.523 ],
       [0.    , 0.    , 1.5792, 0.7674],
       [0.    , 0.5426, 0.    , 0.    ],
       [0.242 , 0.    , 0.    , 0.    ],
       [0.    , 0.3142, 0.    , 0.    ],
       [1.4656, 0.    , 0.0675, 0.    ],
       [0.    , 0.1109, 0.    , 0.3757]])

In [112]:
data[names != 'Joe'] = 7

In [113]:
data

array([[7.    , 7.    , 7.    , 7.    ],
       [0.    , 0.    , 1.5792, 0.7674],
       [7.    , 7.    , 7.    , 7.    ],
       [7.    , 7.    , 7.    , 7.    ],
       [7.    , 7.    , 7.    , 7.    ],
       [1.4656, 0.    , 0.0675, 0.    ],
       [0.    , 0.1109, 0.    , 0.3757]])

Note:

> Selecting data from an array by boolean indexing always creates a copy of the data,
even if the returned array is unchanged.

---

***Practice***
Create a copy of `data` named `data2`, and replace negative values with -1 and positive values with +1.

In [114]:
np.random.seed(42)
data = np.random.randn(7, 4)

In [115]:
data

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249, -0.5623],
       [-1.0128,  0.3142, -0.908 , -1.4123],
       [ 1.4656, -0.2258,  0.0675, -1.4247],
       [-0.5444,  0.1109, -1.151 ,  0.3757]])

In [116]:
data2 = data.copy()

In [117]:
data2 < 0

array([[False,  True, False, False],
       [ True,  True, False, False],
       [ True, False,  True,  True],
       [False,  True,  True,  True],
       [ True, False,  True,  True],
       [False,  True, False,  True],
       [ True, False,  True, False]])

In [118]:
data2[data2 < 0] = -1

In [119]:
data2

array([[ 0.4967, -1.    ,  0.6477,  1.523 ],
       [-1.    , -1.    ,  1.5792,  0.7674],
       [-1.    ,  0.5426, -1.    , -1.    ],
       [ 0.242 , -1.    , -1.    , -1.    ],
       [-1.    ,  0.3142, -1.    , -1.    ],
       [ 1.4656, -1.    ,  0.0675, -1.    ],
       [-1.    ,  0.1109, -1.    ,  0.3757]])

In [120]:
data2[data2 > 0] = 1

In [121]:
data2

array([[ 1., -1.,  1.,  1.],
       [-1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.],
       [ 1., -1., -1., -1.],
       [-1.,  1., -1., -1.],
       [ 1., -1.,  1., -1.],
       [-1.,  1., -1.,  1.]])

What if we a one-line solution?
We should be able to use `np.where()`, which is the NumPy version of Excel's `if()`.

In [122]:
data3 = np.where(data < 0, -1, +1)

A possible problem with our `np.where()` solution is that we want to keep 0s as 0s.
NumPy's `np.select()` is replacement for nested `np.where()` statements.

In [123]:
data4 = np.select(
    condlist=[data < 0, data > 0],
    choicelist=[-1, +1]
)

In [124]:
data2 == data3

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [125]:
data2 == data4

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

---

## Transposing Arrays and Swapping Axes

In [126]:
np.arange(15)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [127]:
np.arange(15).reshape((3, 5), order='C')

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [128]:
np.arange(15).reshape((3, 5), order='F')

array([[ 0,  3,  6,  9, 12],
       [ 1,  4,  7, 10, 13],
       [ 2,  5,  8, 11, 14]])

In [129]:
arr = np.arange(15).reshape((3, 5))

The `.T` method transposes the array `arr`.

In [130]:
arr.T

array([[ 0,  5, 10],
       [ 1,  6, 11],
       [ 2,  7, 12],
       [ 3,  8, 13],
       [ 4,  9, 14]])

In [131]:
np.random.seed(42)
arr = np.random.randn(6, 3)

In [132]:
arr

array([[ 0.4967, -0.1383,  0.6477],
       [ 1.523 , -0.2342, -0.2341],
       [ 1.5792,  0.7674, -0.4695],
       [ 0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249],
       [-0.5623, -1.0128,  0.3142]])

In [133]:
np.dot(arr.T, arr)

array([[ 5.7293,  0.6418, -1.623 ],
       [ 0.6418,  5.5641,  2.8028],
       [-1.623 ,  2.8028,  3.9857]])

Review the docstring for `np.dot()`, which we will use extensively for portfolio optimization.

---

***Practice:***
Create a 3-by-3 array named `a4` that counts from 1 to 9 in the rows.

In [134]:
np.arange(1, 10).reshape((3, 3))

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

***Practice:***
Create a 3-by-3 array named `a5` that counts from 9 to 1 in the columns.

In [135]:
np.arange(1, 10).reshape((3, 3), order='F')

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

***Practice:***
How do `.reshape()` and `.resize()` differ?

`.reshape()` returns a reshaped array, whereas `.resize()` reshapes the array *inplace*.
Also, `.reshape()` cannot add or drop elements while reshaping, whereas `.resize()` allows us to add or drop elements while reshaping.
***ALWAYS READ THE DOCSTRING AND CHECK YOUR OUTPUT!***

In [136]:
_ = np.arange(1, 10)
_.resize((3, 2))
_

array([[1, 2],
       [3, 4],
       [5, 6]])

In [137]:
_ = np.arange(1, 10)
_.resize((3, 4))
_

array([[1, 2, 3, 4],
       [5, 6, 7, 8],
       [9, 0, 0, 0]])

---

# Universal Functions: Fast Element-Wise Array Functions

> A universal function, or ufunc, is a function that performs element-wise operations on data in ndarrays. You can think of them as fast vectorized wrappers for simple functions that take one or more scalar values and produce one or more scalar results.

In [138]:
arr = np.arange(10)

In [139]:
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [140]:
np.sqrt(arr)

array([0.    , 1.    , 1.4142, 1.7321, 2.    , 2.2361, 2.4495, 2.6458,
       2.8284, 3.    ])

Here `np.exp(x)` is $e^x$.

In [141]:
np.exp(arr)

array([   1.    ,    2.7183,    7.3891,   20.0855,   54.5982,  148.4132,
        403.4288, 1096.6332, 2980.958 , 8103.0839])

In [142]:
2**arr

array([  1,   2,   4,   8,  16,  32,  64, 128, 256, 512])

> These are referred to as unary ufuncs. Others, such as add or maximum, take two arrays (thus, binary ufuncs) and return a single array as the result.

In [143]:
np.random.seed(42)
x = np.random.randn(8)
y = np.random.randn(8)

In [144]:
np.maximum(x, y)

array([ 0.4967,  0.5426,  0.6477,  1.523 ,  0.242 , -0.2341,  1.5792,
        0.7674])

***Be careful! Function names are not the whole story. Check your output and read the docstring!***

In [145]:
np.max(x)

1.5792128155073915

***Table 4-3*** lists the fast, element-wise unary functions:

- `abs`, `fabs`: Compute the absolute value element-wise for integer, oating-point, or complex values
- `sqrt`: Compute the square root of each element (equivalent to arr ** 0.5) 
- `square`: Compute the square of each element (equivalent to arr ** 2)
- `exp`: Compute the exponent $e^x$ of each element
- `log`, `log10`, `log2`, `log1p`: Natural logarithm (base e), log base 10, log base 2, and log(1 + x), respectively
- `sign`: Compute the sign of each element: 1 (positive), 0 (zero), or –1 (negative)
- `ceil`: Compute the ceiling of each element (i.e., the smallest integer greater than or equal to thatnumber)
- `floor`: Compute the oor of each element (i.e., the largest integer less than or equal to each element)
- `rint`: Round elements to the nearest integer, preserving the dtype
- `modf`: Return fractional and integral parts of array as a separate array
- `isnan`: Return boolean array indicating whether each value is NaN (Not a Number)
- `isfinite`, `isinf`: Return boolean array indicating whether each element is finite (non-inf, non-NaN) or infinite, respectively
- `cos`, `cosh`, `sin`, `sinh`, `tan`, `tanh`: Regular and hyperbolic trigonometric functions
- `arccos`, `arccosh`, `arcsin`, `arcsinh`, `arctan`, `arctanh`: Inverse trigonometric functions
- `logical_not`: Compute truth value of not x element-wise (equivalent to ~arr).

***Table 4-4*** lists the fast, element-wise binary functions:

- `add`: Add corresponding elements in arrays
- `subtract`: Subtract elements in second array from first array
- `multiply`: Multiply array elements
- `divide`, `floor_divide`: Divide or floor divide (truncating the remainder)
- `power`: Raise elements in first array to powers indicated in second array
- `maximum`, `fmax`: Element-wise maximum; `fmax` ignores `NaN`
- `minimum`, `fmin`: Element-wise minimum; `fmin` ignores `NaN`
- `mod`: Element-wise modulus (remainder of division)
- `copysign`: Copy sign of values in second argument to values in first argument
- `greater`, `greater_equal`, `less`, `less_equal`, `equal`, `not_equal`: Perform element-wise comparison, yielding boolean array (equivalent to infix operators >, >=, <, <=, ==, !=)
- `logical_and`, `logical_or`, `logical_xor`: Compute element-wise truth value of logical operation (equivalent to infix operators & |, ^)

# Array-Oriented Programming with Arrays

> Using NumPy arrays enables you to express many kinds of data processing tasks as concise array expressions that might otherwise require writing loops. This practice of replacing explicit loops with array expressions is commonly referred to as vectorization. In general, vectorized array operations will often be one or two (or more) orders of magnitude faster than their pure Python equivalents, with the biggest impact in any kind of numerical computations. Later, in Appendix A, I explain broadcasting, a powerful method for vectorizing computations.

## Expressing Conditional Logic as Array Operations

> The numpy.where function is a vectorized version of the ternary expression x if condition else y.

In [146]:
xarr = np.array([1.1, 1.2, 1.3, 1.4, 1.5])
yarr = np.array([2.1, 2.2, 2.3, 2.4, 2.5])
cond = np.array([True, False, True, True, False])

In [147]:
result = [(x if c else y) for x, y, c in zip(xarr, yarr, cond)]

In [148]:
result

[1.1, 2.2, 1.3, 1.4, 2.5]

NumPy's `where()` is an if-else statement that operates like Excel's `if()`.

---

***Practice:***
Replicate the list comprehension above with `np.where()`.

In [149]:
np.where(cond)

(array([0, 2, 3]),)

In [150]:
np.where(cond, xarr, yarr)

array([1.1, 2.2, 1.3, 1.4, 2.5])

***Practice:***
Replicate the list comprehension above with `np.select()`.

I like the following solution because it is very explicit:

In [151]:
np.select(
    condlist=[cond==True, cond==False],
    choicelist=[xarr, yarr]
)

array([1.1, 2.2, 1.3, 1.4, 2.5])

In [152]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [153]:
np.select(
    condlist=[cond, ~cond],
    choicelist=[xarr, yarr]
)

array([1.1, 2.2, 1.3, 1.4, 2.5])

---

## Mathematical and Statistical Methods

> A set of mathematical functions that compute statistics about an entire array or about the data along an axis are accessible as methods of the array class. You can use aggregations (often called reductions) like sum, mean, and std (standard deviation) either by calling the array instance method or using the top-level NumPy function.

We will use these aggregations extensively in pandas.

In [154]:
np.random.seed(42)
arr = np.random.randn(5, 4)

In [155]:
arr

array([[ 0.4967, -0.1383,  0.6477,  1.523 ],
       [-0.2342, -0.2341,  1.5792,  0.7674],
       [-0.4695,  0.5426, -0.4634, -0.4657],
       [ 0.242 , -1.9133, -1.7249, -0.5623],
       [-1.0128,  0.3142, -0.908 , -1.4123]])

In [156]:
arr.mean()

-0.17129856144182892

In [157]:
np.mean(arr)

-0.17129856144182892

In [158]:
arr.sum()

-3.4259712288365787

The aggregation methods above aggregated the whole array.
We can use the `axis` argument to aggregate columns (`axis=0`) and rows (`axis=1`).

In [159]:
arr.mean(axis=1)

array([ 0.6323,  0.4696, -0.214 , -0.9896, -0.7547])

In [160]:
arr[0].mean()

0.6322920615871914

In [161]:
arr.mean(axis=0)

array([-0.1956, -0.2858, -0.1739, -0.03  ])

In [162]:
arr = np.array([0, 1, 2, 3, 4, 5, 6, 7])
arr.cumsum()

array([ 0,  1,  3,  6, 10, 15, 21, 28])

In [163]:
arr = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])

In [164]:
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [165]:
arr.cumsum(axis=0)

array([[ 0,  1,  2],
       [ 3,  5,  7],
       [ 9, 12, 15]])

In [166]:
arr.cumprod(axis=1)

array([[  0,   0,   0],
       [  3,  12,  60],
       [  6,  42, 336]])

Table 4-5 lists the basic statistical methods:

- `sum`: Sum of all the elements in the array or along an axis; zero-length arrays have sum 0
- `mean`: Arithmetic mean; zero-length arrays have NaN mean
- `std`, `var`: Standard deviation and variance, respectively, with optional degrees of freedom adjustment (default denominator $n$)
- `min`, `max`: Minimum and maximum
- `argmin`, `argmax`: Indices of minimum and maximum elements, respectively
- `cumsum`: Cumulative sum of elements starting from 0
- `cumprod`: Cumulative product of elements starting from 1

NumPy's `.var()` and `.std()` methods return *population* statistics (denominators of $n$).
The pandas equivalents return *sample* statistics (denominators of $n-1$), which are more appropriate for financial data analysis where we have a sample instead of a population.

## Methods for Boolean Arrays

In [167]:
np.random.seed(42)
arr = np.random.randn(100)

In [168]:
(arr > 0).sum() # Number of positive values

46

In [169]:
(arr > 0).mean() # percentage of positive values

0.46

In [170]:
bools = np.array([False, False, True, False])

In [171]:
bools.any()

True

In [172]:
bools.all()

False

## Sorting

> Like Python's built-in list type, NumPy arrays can be sorted in-place with the sort method.

In [173]:
np.random.seed(42)
arr = np.random.randn(6)

In [174]:
arr.sort()

In [175]:
np.random.seed(42)
arr = np.random.randn(5, 3)

For two-dimensional arrays (and beyond), we can sort along an axis.
The default sort axis is the last axis.
Recall that `axis=1` operates on rows.
Note that these row sorts are independent of one another.

In [176]:
arr.sort(1)